# 1. 데이터 로드 및 전처리

In [2]:
import pandas as pd
import altair as alt
from ipywidgets import interact, widgets
from vega_datasets import data

# Altair 데이터 변환 설정
alt.data_transformers.disable_max_rows()

param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [3]:
all_data = pd.read_csv('data.csv')

In [4]:
all_data.shape

(36742, 18)

# 2. Heatmap 출력

interactive(children=(Dropdown(description='Parameter:', options=('Precipitation (mm)', 'Number of Days with P…

<function __main__.update(selected_param, selected_month)>

In [ ]:


# Altair 세계 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

# 파라미터 선택 위젯
param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Precipitation (mm)',
    description='Parameter:'
)

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jan',
    description='Month:'
)

# 필터링 슬라이더
slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=500,
    step=1,
    description='Filter:',
    layout=widgets.Layout(width='80%')
)

# Output 위젯
output = Output()

# 최소/최대값 계산 함수
def calculate_min_max(selected_param, selected_month):
    filtered = all_data[all_data['Parameter'] == selected_param]
    min_val = filtered[selected_month].min()
    max_val = filtered[selected_month].max()
    return min_val, max_val

# 슬라이더 업데이트 함수
def update_slider(selected_param, selected_month):
    min_val, max_val = calculate_min_max(selected_param, selected_month)
    slider.min = min_val
    slider.max = max_val
    slider.value = [min_val, max_val]

# Heatmap 생성 함수
def plot_heatmap(selected_param, selected_month):
    filtered = all_data[all_data['Parameter'] == selected_param]

    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    heatmap = alt.Chart(filtered).mark_circle().encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        color=alt.Color(f'{selected_month}:Q', scale=alt.Scale(scheme='viridis'), title='Value'),
        size=alt.Size(f'{selected_month}:Q', title='Intensity'),
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title=selected_param)
        ]
    )

    return background + heatmap

# 깃발 추가 함수
def add_flags(selected_param, selected_month, value_range):
    min_val, max_val = value_range
    filtered = all_data[
        (all_data['Parameter'] == selected_param) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    points = alt.Chart(filtered).mark_text(text="🚩", size=15, color='red').encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return points

# 업데이트 함수
def update_plot(change=None):
    with output:
        # Output 위젯 초기화
        output.clear_output()
        
        # 현재 선택 상태
        selected_param = param_dropdown.value
        selected_month = month_dropdown.value
        value_range = slider.value

        # Heatmap 생성
        heatmap_chart = plot_heatmap(selected_param, selected_month)

        # 깃발 추가
        flag_chart = add_flags(selected_param, selected_month, value_range)

        # 두 차트 결합
        chart = heatmap_chart + flag_chart

        # 차트 표시
        display(chart)

# 이벤트 핸들러 등록
param_dropdown.observe(update_plot, names='value')
month_dropdown.observe(update_plot, names='value')
slider.observe(update_plot, names='value')

# 슬라이더 동적 업데이트
def update_slider_on_change(change):
    selected_param = param_dropdown.value
    selected_month = month_dropdown.value
    update_slider(selected_param, selected_month)

param_dropdown.observe(update_slider_on_change, names='value')
month_dropdown.observe(update_slider_on_change, names='value')

# UI 표시
ui = widgets.VBox([param_dropdown, month_dropdown, slider])
display(ui, output)

# 초기 차트 표시
update_plot()

Output()